## Import the required libraries

In [ ]:
import pandas as pd
import krakenex
from pandas import json_normalize
import datetime as dt
from datetime import date
from datetime import timezone
from datetime import timedelta
import pygsheets
import time
import numpy as np

## Authentitication

In [ ]:
k = krakenex.API()
k.load_key('kraken.key')

## Call the positions

In [ ]:
current_pf_status = k.query_private('Balance')
current_pf_status = current_pf_status['result']
current_pf_status = pd.DataFrame.from_dict(current_pf_status, orient='index').reset_index()
current_pf_status = current_pf_status.rename(columns={"index": "crypto", 0: "value"})
current_pf_status['time'] = dt.date.today()
current_pf_status

## Get the list of transactions all time

In [ ]:
def add_one_month(t):
    one_day = dt.timedelta(days=1)
    one_month_later = t + one_day
    while one_month_later.month == t.month:  # advance to start of next month
        one_month_later += one_day
    target_month = one_month_later.month
    while one_month_later.day < t.day:  # advance to appropriate day
        one_month_later += one_day
        if one_month_later.month != target_month:  # gone too far
            one_month_later -= one_day
            break
    return one_month_later

start = dt.date(2021, 1, 1)

#let's create the master dataset that will list all transactions
transactions_column_names = ['refid', 'time', 'type', 'subtype', 'aclass', 'asset', 'amount', 'fee', 'balance']
transactions_df = pd.DataFrame(columns = transactions_column_names)

while start < dt.date.today():
    
    #create the end date one month later
    end = add_one_month(start)
    #format the dates into unix for the api parameters
    formatted_start = time.mktime(start.timetuple())
    formatted_end = time.mktime(end.timetuple())
    #change the start date for the next loop
    start = end
    
    #let's call the endpoint to have all transaction details in EUR
    req_data = {'aclass': 'ZEUR',
                'start': formatted_start,
                'end': formatted_end
                }
    transactions = k.query_private('Ledgers', req_data)
    
    if transactions.get('error'):
        pass
    
    elif transactions['result']['ledger'] is None:
        pass
    
    else:
        transactions = transactions['result']['ledger']
        transactions = json_normalize(transactions)
        
        #let's pull out the list of all ledger IDs
        ledger_list = list(transactions.columns)
        sep = '.'
        ledger_list = [i.split(sep, 1)[0] for i in ledger_list]
        clean_ledger_list=[]
        for i in ledger_list:
          if i not in clean_ledger_list:
            clean_ledger_list.append(i)
        
        #let's create the temporary dataset that will list all transactions
        temp_df = pd.DataFrame(columns = transactions_column_names)
        
        #let's loop over the list of ledgers to get all the transactions and add them to the master dataframe
        all_transactions = k.query_private('Ledgers', req_data)
        for i in clean_ledger_list:
            transaction_row = all_transactions['result']['ledger'][i]
            transaction_row = json_normalize(transaction_row)
            temp_df = temp_df.append(transaction_row)
    transactions_df = transactions_df.append(temp_df)
transactions_df

## Transform the dataframe to crypto Vs. euro rows

In [ ]:
deposit = transactions_df
deposit['fee'] =  deposit['fee'].apply(lambda x: float(x))
deposit = deposit[deposit['type'] == 'deposit']
deposit = deposit.filter(items=['time', 'refid', 'fee', 'amount'])
deposit['time'] = pd.to_datetime(deposit['time'],unit='s')
deposit

## Calculate the internal flows

In [ ]:
internal_flows = transactions_df
internal_flows = internal_flows[internal_flows['type'] == 'spend']
internal_flows = internal_flows.filter(items=['time', 'refid', 'asset', 'amount'])
internal_flows['time'] = pd.to_datetime(internal_flows['time'],unit='s')
internal_flows = internal_flows[internal_flows['asset'] != 'EUR.HOLD']
internal_flows

## Calculate the incoming crypto flows

In [ ]:
crypto_positive = transactions_df
crypto_positive = crypto_positive[crypto_positive['type'] == 'receive']
crypto_positive = crypto_positive.filter(items=['time', 'refid', 'asset', 'amount'])
crypto_positive['time'] = pd.to_datetime(crypto_positive['time'],unit='s')
crypto_positive

## Create a daily dataframe containing all portfolio transactions regardless of the crypto or currency flow

In [ ]:
ongoing_portfolio = crypto_positive.append(internal_flows)
ongoing_portfolio['time'] = ongoing_portfolio['time'].dt.strftime('%Y-%m-%d')
ongoing_portfolio['time'] = pd.to_datetime(ongoing_portfolio['time'], format='%Y-%m-%d')
ongoing_portfolio = ongoing_portfolio.filter(items=['time', 'asset', 'amount'])

# create a range of dates for the merged dataframe
index_of_dates = pd.date_range('2020-01-01', date.today()).to_frame().reset_index(drop=True).rename(columns={0: 'time'})

# create a merged dataframe with date / asset / transactions 
ongoing_portfolio = pd.merge(index_of_dates,ongoing_portfolio,how='left', on='time')
ongoing_portfolio = ongoing_portfolio.rename(columns={'asset': 'crypto'})
ongoing_portfolio

## Add the value of the crypto to the dataframe

In [ ]:
#get the list of all crypto that are available in the portfolio
crypto_list = ongoing_portfolio['crypto'].replace({'XMLN':'XMLNZ', 'XXDG':'XDG'}).dropna().unique()
crypto_list = crypto_list + 'EUR'
crypto_list

#set up an empty crypto df
crypto_value_columns = ['time','crypto', 'crypto_value']
crypto_value_df = pd.DataFrame(columns = crypto_value_columns)

for i in crypto_list:
    
    #set up the start date of the loop
    start = dt.date(2021, 1, 1)
    formatted_start = time.mktime(start.timetuple())
    
    #let's call the endpoint to have all transaction details in EUR
    req_data = {'pair': i,
                'interval': '1440',
                'since': formatted_start
                }
    crypto_value = k.query_public('OHLC', req_data)
    
    if crypto_value.get('error'):
        pass
    
    elif crypto_value['result'][i] is None:
        pass
    
    else:
        crypto_value = crypto_value['result'][i]
        crypto_value = pd.DataFrame.from_records(crypto_value)
        crypto_value[0] = pd.to_datetime(crypto_value[0],unit='s')
        crypto_value['crypto'] = i
        crypto_value = crypto_value.rename(columns={0: "time", 1: "crypto_value"})
        crypto_value = crypto_value.filter(items=['time', 'crypto', 'crypto_value'])
    
    crypto_value_df = crypto_value_df.append(crypto_value, ignore_index=True)

crypto_value_df['time'] = pd.to_datetime(crypto_value_df['time'])
crypto_value_df['crypto'] = crypto_value_df['crypto'].apply(lambda x: str(x)[:-3])
crypto_value_df['crypto'] = crypto_value_df['crypto'].replace('XMLNZ', 'XMLN')
crypto_value_df['crypto'] = crypto_value_df['crypto'].replace('XDG', 'XXDG')
crypto_value_df = crypto_value_df.drop(columns=['error'])
crypto_value_df

## Join to the current pf value

In [ ]:
last_crypto_value =  current_pf_status.set_index(['time', 'crypto']).join(crypto_value_df.set_index(['time', 'crypto'])).reset_index()
last_crypto_value['value'] = last_crypto_value['value'].apply(lambda x: float(x))
last_crypto_value['crypto_value'] = last_crypto_value['crypto_value'].apply(lambda x: float(x))
last_crypto_value['total_crypto_value'] = last_crypto_value['value']*last_crypto_value['crypto_value']
last_crypto_value = last_crypto_value.replace(np.nan, '')
last_crypto_value = last_crypto_value[last_crypto_value['total_crypto_value'] != '']
last_crypto_value

In [ ]:
#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[2]

#update the required sheet
wks.set_dataframe(last_crypto_value,(1,1))

## Join to the ongoing portfolio with the crypto values

In [ ]:
final_df = crypto_value_df.set_index(['time', 'crypto']).join(ongoing_portfolio.set_index(['time', 'crypto'])).reset_index()
final_df = final_df.drop(columns=['crypto_value'])
final_df['amount'] = final_df['amount'].apply(lambda x: float(x))
final_df = final_df.replace(np.nan, 0)
final_df['cum_amount'] = final_df.groupby(['crypto']).amount.cumsum()

final_df = final_df[final_df['cum_amount'] != 0]
final_df['time'] = pd.to_datetime(final_df['time'])
final_df = final_df.set_index(['time', 'crypto']).join(crypto_value_df.set_index(['time', 'crypto'])).reset_index()
final_df['cum_amount'] = final_df['cum_amount'].apply(lambda x: float(x))
final_df['crypto_value'] = final_df['crypto_value'].apply(lambda x: float(x))
final_df['cum_amount_eur_value'] = final_df['cum_amount']*final_df['crypto_value']
final_df = final_df.filter(items=['time', 'crypto', 'crypto_value', 'amount', 'cum_amount', 'cum_amount_eur_value'])
final_df

In [ ]:
#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[3]

#update the required sheet
wks.set_dataframe(final_df,(1,1))

## Send the raw transaction df

In [ ]:
raw_dataframe = transactions_df
raw_dataframe['time'] = pd.to_datetime(raw_dataframe['time'],unit='s')
raw_dataframe['time'] = raw_dataframe['time'].dt.strftime('%Y-%m-%d')
raw_dataframe = raw_dataframe.filter(items=['refid', 'time', 'asset', 'amount', 'fee', 'balance'])

#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[0]

#update the required sheet
wks.set_dataframe(raw_dataframe,(1,1))

## Add aggregated portfolio value to the deposit df 

In [ ]:
#create a dataframe with euro spending
eur_spending = deposit
eur_spending = eur_spending.filter(items=['time', 'amount']).rename(columns={"amount": "eur_spent"})
eur_spending['eur_spent'] = eur_spending['eur_spent'].apply(lambda x: float(x))
eur_spending = eur_spending.groupby(['time']).sum().reset_index()
eur_spending['time'] = pd.to_datetime(eur_spending['time']).dt.strftime('%Y-%m-%d')

#create amn aggregated dataframe of crypto values
aggregated_df = final_df
aggregated_df = aggregated_df.groupby(['time']).sum().reset_index()
aggregated_df['time'] = pd.to_datetime(aggregated_df['time']).dt.strftime('%Y-%m-%d')

#combine both dataframes to have both spending and porftolio value
aggregated_df = aggregated_df.set_index('time').join(eur_spending.set_index('time')).reset_index()
aggregated_df['eur_spent'] = aggregated_df['eur_spent'].replace(np.nan, 0)
aggregated_df['eur_spent'] = aggregated_df['eur_spent'].apply(lambda x: float(x))
aggregated_df['cum_eur_spent'] = aggregated_df.eur_spent.cumsum()
aggregated_df

In [ ]:
#authorization
gc = pygsheets.authorize(service_file='kraken-reporting-key-sheets.json')

#open the google spreadsheetkraken-credentials
sh = gc.open("Kraken reporting")

#select the first sheet
wks = sh[1]

#update the required sheet
wks.set_dataframe(aggregated_df,(1,1))